In [1]:
# Standard libraries
import xarray as xr
import numpy as np
import pandas as pd
# import os
# from glob import glob
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
%matplotlib inline
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import seaborn as sns
import iris
from iris.pandas import as_cubes
import sys

from datetime import datetime
from cartopy.util import add_cyclic_point
import gc
import imageio.v2
from IPython import display
# import netCDF4
from global_land_mask import globe
# # Import tobac itself:
import tobac

# Disable a few warnings:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, append=True)
warnings.filterwarnings('ignore', category=RuntimeWarning, append=True)
warnings.filterwarnings('ignore', category=FutureWarning, append=True)
warnings.filterwarnings('ignore',category=pd.io.pytables.PerformanceWarning)

In [2]:
%%time
path = '/glade/u/home/noteng/work/research/data/'
file = 'march13-march14.nc'
data = xr.open_dataset(path+file)
data = data.sel(time=slice('2020-03-13T04:00:00.000000000', '2020-03-14T05:00:00.000000000'))
data.close()

CPU times: user 26.4 ms, sys: 11.7 ms, total: 38.1 ms
Wall time: 438 ms


In [3]:
# equivalent_reflectivity_factor = data['equivalent_reflectivity_factor'][:,450:580,256:771] #Based on longitude and latitude of Andoya and Norwegian Sea
equivalent_reflectivity_factor = data['equivalent_reflectivity_factor'][:,250:650,450:850] #Based on longitude and latitude of Andoya and Norwegian Sea
# equivalent_reflectivity_factor = data['equivalent_reflectivity_factor'][:,330:580,660:780] #### hdm1 and hdm2
# equivalent_reflectivity_factor = data['equivalent_reflectivity_factor']
equivalent_reflectivity_factor

<xarray.DataArray 'equivalent_reflectivity_factor' (time: 301, Yc: 400, Xc: 400)>
[48160000 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 2020-03-13T04:00:00 ... 2020-03-14T05:00:00
  * Xc       (Xc) float32 -8.3e+04 -8.2e+04 -8.1e+04 ... 3.15e+05 3.16e+05
  * Yc       (Yc) float32 -2.06e+06 -2.061e+06 ... -2.458e+06 -2.459e+06
    lon      (Yc, Xc) float32 ...
    lat      (Yc, Xc) float32 ...
Attributes:
    standard_name:  equivalent_reflectivity_factor
    long_name:      Radar Reflectivity
    units:          dBZ
    grid_mapping:   projection_lambert

In [4]:
# load Mask_ERF_files
Mask_ERF = iris.load('../saved-files/threshold-5/Mask_ERF_iris-5.nc')[0]
Mask_ERF

<iris 'Cube' of segmentation_mask / (1) (time: 301; -- : 400; -- : 400)>

In [5]:
track = pd.read_csv('../saved-files/threshold-5/track-reset-5.csv')
track

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell
0,0,1,1.000000,337.000000,1,5,1,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.061000e+06,254000.000000,71.319077,17.025770,1,0 days 00:00:00
1,0,2,84.541826,399.000000,4,5,2,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.144542e+06,316000.000000,70.491052,18.382262,2,0 days 00:00:00
2,0,3,99.331990,249.412318,8,5,3,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.159332e+06,166412.318038,70.508909,14.406879,3,0 days 00:00:00
3,0,4,100.000000,256.000000,1,5,4,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.160000e+06,173000.000000,70.498161,14.579193,4,0 days 00:00:00
4,0,5,102.712706,388.653061,11,5,5,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.162713e+06,305653.061269,70.340917,18.044254,5,0 days 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23081,300,31,282.691712,343.573782,48,5,23078,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.342692e+06,260573.782460,68.765705,16.346828,16269,0 days 00:00:00
23082,300,32,293.244980,386.256512,167,5,23079,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.353245e+06,303256.512311,68.623712,17.343087,16270,0 days 00:00:00
23083,300,33,292.148658,398.356878,5,5,23080,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.352149e+06,315356.877698,68.619241,17.636212,16271,0 days 00:00:00
23084,300,36,314.820269,272.000000,2,5,23083,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.374820e+06,189000.000000,68.535898,14.550293,16272,0 days 00:00:00


In [6]:
# # Changing time cell to timedelta
# time_cell = track['time_cell'].astype('timedelta64[ns]').values
# time_cell

### CALCULATING FOR DISTANCE

In [7]:
distance = []
for i in range(len(track)):
    #lat_andenes = 69.141599 # latitude of Andenes
    lat_andenes = 69.2412 # latiude of the radar
    #lon_andenes = 15.684104 # latitude of Andenes
    lon_andenes = 16.0030 # longitude of where the radar is located
    
# 69.2412° N, 16.0030° E
    
    distances = tobac.analysis.haversine(track['latitude'][i], track['longitude'][i], lat_andenes, lon_andenes)
    distance.append(distances)

In [8]:
distance[:10]
# np.max(distance)

[234.4631862426416,
 166.31196244899817,
 153.7777890866078,
 150.16790095021747,
 145.40422837751964,
 163.18167914747215,
 127.15958990695803,
 141.83580435010919,
 125.40281715632239,
 148.7877111945513]

In [9]:
%%time
# apped distance to track dataframe
track['distance'] = pd.Series(distance)

CPU times: user 3.68 ms, sys: 0 ns, total: 3.68 ms
Wall time: 3.69 ms


In [10]:
track.head()

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance
0,0,1,1.000000,337.000000,1,5,1,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.061000e+06,254000.000000,71.319077,17.025770,1,0 days 00:00:00,234.463186
1,0,2,84.541826,399.000000,4,5,2,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.144542e+06,316000.000000,70.491052,18.382262,2,0 days 00:00:00,166.311962
2,0,3,99.331990,249.412318,8,5,3,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.159332e+06,166412.318038,70.508909,14.406879,3,0 days 00:00:00,153.777789
3,0,4,100.000000,256.000000,1,5,4,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.160000e+06,173000.000000,70.498161,14.579193,4,0 days 00:00:00,150.167901
4,0,5,102.712706,388.653061,11,5,5,2020-03-13 04:00:00,2020-03-13 04:00:00,-2.162713e+06,305653.061269,70.340917,18.044254,5,0 days 00:00:00,145.404228


## _cells that lived for at least 10 mins old_ 

In [11]:
%%time
# cell_one_hour = np.timedelta64( 600, 's') #10 minutes in seconds
cell_times = np.timedelta64( 600, 's') #10 minutes in seconds

#convert time_cell to seconds
time_cell = track['time_cell'].astype('timedelta64[ns]').values

# track cell that live for at least 10 minutes long
track_close_to_AMF1 = track[ time_cell >= cell_times ]
# track_close_to_AMF1 = track[ ((track['distance'] <= 95) & (track['time_cell'].values >= cell_one_hour)) ]
track_close_to_AMF1= track_close_to_AMF1.drop_duplicates(subset=['cell']) # remove duplicates of cell_ids
track_close_to_AMF1.head()

CPU times: user 47.3 ms, sys: 0 ns, total: 47.3 ms
Wall time: 53 ms


,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance
430,6,37,203.303419,302.324596,49,5,627,2020-03-13 04:30:00,2020-03-13 04:30:00,-2.263303e+06,219324.595821,69.524381,15.534945,416,0 days 00:10:00,36.472873
441,6,44,234.271022,215.148310,201,5,634,2020-03-13 04:30:00,2020-03-13 04:30:00,-2.294271e+06,132148.309768,69.304188,13.296552,425,0 days 00:10:00,106.848586
510,7,7,126.330530,264.496168,48,5,707,2020-03-13 04:35:00,2020-03-13 04:35:00,-2.186331e+06,181496.168102,70.253160,14.745474,487,0 days 00:10:00,122.622611
517,7,11,133.933797,384.825058,6,5,711,2020-03-13 04:35:00,2020-03-13 04:35:00,-2.193934e+06,301825.058283,70.064374,17.833156,492,0 days 00:10:00,115.811544
524,7,12,143.158009,374.575521,43,5,712,2020-03-13 04:35:00,2020-03-13 04:35:00,-2.203158e+06,291575.521481,69.993651,17.538962,497,0 days 00:10:00,102.765114


In [12]:
# total track cells found.
len(track_close_to_AMF1) 

1466

# tracking all cells that were at least 10 minutes old back in time to get all the cells

In [13]:
%%time
combined_tracks_AMF1 = []
for index, cell_id in enumerate(np.unique(track_close_to_AMF1['cell'].values)): # looped through tracks found with AMF1 site
    if cell_id in track['cell']: # checking if those cells(tracks) are within the original tracked dataframe
        track_i = track[track['cell'] == cell_id] # display it if it is within the tracked dataframe
        combined_tracks_AMF1.append(track_i) # append the results you get a new dataFrame calld combined_tracks_AMF1

# Concatenate the dataframes(combined_tracks_AMD) along the rows axis with keys
# tracks_closer_AMF1 = pd.concat(combined_tracks_AMF1, keys=['df1', 'df2', 'df3', 'df4', 'df5', 'df6'])
tracks_closer_AMF1 = pd.concat(combined_tracks_AMF1)
tracks_closer_AMF1

CPU times: user 601 ms, sys: 16.1 ms, total: 617 ms
Wall time: 645 ms


,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance
428,4,28,201.219736,300.689233,11,5,423,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.261220e+06,217689.233099,69.544704,15.498959,416,0 days 00:00:00,39.132921
429,5,38,202.273236,301.660422,35,5,527,2020-03-13 04:25:00,2020-03-13 04:25:00,-2.262273e+06,218660.421950,69.534304,15.520784,416,0 days 00:05:00,37.704661
430,6,37,203.303419,302.324596,49,5,627,2020-03-13 04:30:00,2020-03-13 04:30:00,-2.263303e+06,219324.595821,69.524381,15.534945,416,0 days 00:10:00,36.472873
439,4,37,230.460885,211.799724,187,5,432,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.290461e+06,128799.724015,69.340584,13.218530,425,0 days 00:00:00,110.156057
440,5,44,231.873681,213.475405,183,5,533,2020-03-13 04:25:00,2020-03-13 04:25:00,-2.291874e+06,130475.405470,69.326868,13.258308,425,0 days 00:05:00,108.487906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23027,299,27,271.955203,288.680268,159,5,23039,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.331955e+06,205680.267926,68.912816,15.040489,16225,0 days 00:05:00,52.916488
23028,300,30,272.843216,289.783693,142,5,23077,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.332843e+06,206783.692918,68.903864,15.065471,16225,0 days 00:10:00,52.910819
23031,298,34,296.371715,217.621195,30,5,23006,2020-03-14 04:50:00,2020-03-14 04:50:00,-2.356372e+06,134621.195276,68.737655,13.269794,16228,0 days 00:00:00,122.630657
23032,299,30,296.984193,220.121651,26,5,23042,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.356984e+06,137121.651293,68.730769,13.329528,16228,0 days 00:05:00,120.888777


In [14]:
# resetting the index
tracks_closer_AMF1_index_reset = tracks_closer_AMF1.reset_index(drop=True)
tracks_closer_AMF1_index_reset.head()

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance
0,4,28,201.219736,300.689233,11,5,423,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.261220e+06,217689.233099,69.544704,15.498959,416,0 days 00:00:00,39.132921
1,5,38,202.273236,301.660422,35,5,527,2020-03-13 04:25:00,2020-03-13 04:25:00,-2.262273e+06,218660.421950,69.534304,15.520784,416,0 days 00:05:00,37.704661
2,6,37,203.303419,302.324596,49,5,627,2020-03-13 04:30:00,2020-03-13 04:30:00,-2.263303e+06,219324.595821,69.524381,15.534945,416,0 days 00:10:00,36.472873
3,4,37,230.460885,211.799724,187,5,432,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.290461e+06,128799.724015,69.340584,13.218530,425,0 days 00:00:00,110.156057
4,5,44,231.873681,213.475405,183,5,533,2020-03-13 04:25:00,2020-03-13 04:25:00,-2.291874e+06,130475.405470,69.326868,13.258308,425,0 days 00:05:00,108.487906


In [15]:
# len(tracks_closer_AMF1_index_reset)

# Algorithm to calculate the corresponding reflectivity values (calculate for the max and mean dBZ) and the area

In [16]:
unique_cell_ids = track_close_to_AMF1['cell'].values # getting the cells_tracked
len(unique_cell_ids)

1466

In [17]:
# len(np.unique(tracks_closer_AMF1_index_reset['cell']))
# Mask_ERF = iris.load('./saved-files/Mask_ERF_iris.nc')[0]
# Mask_ERF

In [18]:
%%time
# all masked cell in iris cube
masked_tracked_cell = []
for i, unique_value in enumerate(unique_cell_ids):
    # get masked cell based on cell id
    get_mask_cell = tobac.utils.mask_cell(mask=Mask_ERF, cell=unique_value, track=track, masked=False) 
    masked_tracked_cell.append(get_mask_cell)

CPU times: user 4.14 s, sys: 32.9 ms, total: 4.18 s
Wall time: 4.29 s


In [19]:
# masked_tracked_cell[:10]

In [22]:
%%time
# Convert the segmented/mask cells from iris cube to xrray
masked_values_to_xarray = []
for i, items in enumerate(masked_tracked_cell):
    mask_convert_xr = xr.DataArray.from_iris(masked_tracked_cell[i]) # convert iris cube to xarray data array
    masked_values_to_xarray.append(mask_convert_xr) # append the results to list
    # print(mask_convert_xr)

CPU times: user 3.11 s, sys: 43.9 ms, total: 3.15 s
Wall time: 3.33 s


In [23]:
%%time
# Getting the original masked reflectivity values
slice_cell_times = []
for i, cell_id in enumerate(np.unique(tracks_closer_AMF1['cell'])):
    cell_ids = tracks_closer_AMF1_index_reset[tracks_closer_AMF1_index_reset['cell'] == cell_id] 
    first_time = cell_ids['timestr'][::].values[0] # get first time (start time)
    last_time = cell_ids['timestr'][::-1].values[0]  # get the last time value (end time)
    # slice values based on time
    slice_cell_time = masked_values_to_xarray[i].sel(time=slice(first_time, last_time))
    slice_cell_times.append(slice_cell_time)
    # print(slice_cell_time)

CPU times: user 2.29 s, sys: 4.46 ms, total: 2.3 s
Wall time: 2.34 s


In [24]:
# slice_cell_times[:5]

In [25]:
# type(slice_cell_times)

In [26]:
# %%time
# # Create an empty Dataset
# dataset = xr.Dataset()

# # Add each DataArray to the Dataset as a separate variable
# for index, data_array in enumerate(slice_cell_times):
#     var_name = f'data_array_{index}'  # Create a variable name for each DataArray
#     dataset[var_name] = data_array  # Add the DataArray to the Dataset

# # Save the Dataset as a NetCDF file
# dataset.to_netcdf('all_data_arrays.nc')

In [27]:
%%time
equivalent_reflectivity_factors = 10**((equivalent_reflectivity_factor/10))
# equivalent_reflectivity_factors = 10*np.log10(Z)

CPU times: user 4.2 s, sys: 289 ms, total: 4.49 s
Wall time: 4.89 s


## algorithm to extract the speed, area, maximum, minimum and mean reflectivity values based on cell_id and specific times

In [28]:
%%time
min_reflectivities = []; mean_reflectivities = []; max_reflectivities = []; areas = [];
for i, cell_id in enumerate(np.unique(tracks_closer_AMF1['cell'])):
    #################################
    import warnings
    warnings.filterwarnings('ignore')
    #################################
    cell_ids = tracks_closer_AMF1_index_reset[tracks_closer_AMF1_index_reset['cell'] == cell_id]  # dataframe based on cell_id
    first_time = cell_ids['timestr'][::].values[0] # get first time (start time)
    last_time = cell_ids['timestr'][::-1].values[0]  # get the last time value (end time)
    
#     # # # # Calculating area
    # area = tobac.analysis.calculate_area(track, Mask_ERF)
#     # area = tobac.analysis.calculate_area(cell_ids, masked_tracked_cell[i], method_area=None)
#     area = np.absolute(area['area']/1000000) # area in kilometers
#     areas.append(area)
    
    # area1 = np.divide(area['area'],1000000)
    # area['area($km^2$)'] = area1


    # reflectivity_masked based on times
    reflectivity_masked = equivalent_reflectivity_factors.sel(time=slice(first_time, last_time))
    
    # LOOPING THROUGH THE AND GETTING THE VALUES
    for index in (range(len(reflectivity_masked))):
        mean_ref = reflectivity_masked[index].mean().item() # get mean values of reflectivity # in dBZ values
        # # Convert from dBZ to Z values
        # mean_ref_Z = 10**(mean_ref / 10)
        # # Convert it back to dBZ units
        # mean_ref_dBZ = 10*np.log10(mean_ref_Z)
        #####################################
        max_ref = reflectivity_masked[index].max().item() # get max values of reflectivity
        
        mean_reflectivities.append(mean_ref)
        max_reflectivities.append(max_ref)
        
# concatenate the area values to one series
# area1 = pd.concat(areas)

# calculate velocity
# velocity = tobac.analysis.calculate_velocity(tracks_closer_AMF1_index_reset)

CPU times: user 11.2 s, sys: 21.7 ms, total: 11.2 s
Wall time: 11.6 s


In [29]:
# converting the mean Z unit back to dBZ
mean_reflectivities1 = 10*np.log10(mean_reflectivities)

# converting the max Z unit back to dBZ
max_reflectivities1 = 10*np.log10(max_reflectivities)

In [30]:
np.min(mean_reflectivities1), np.max(mean_reflectivities1)

(12.80625991860737, 17.0430868015557)

In [31]:
# area1 = pd.concat(areas)
# len(area1)
tracks_closer_AMF1_index_reset

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance
0,4,28,201.219736,300.689233,11,5,423,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.261220e+06,217689.233099,69.544704,15.498959,416,0 days 00:00:00,39.132921
1,5,38,202.273236,301.660422,35,5,527,2020-03-13 04:25:00,2020-03-13 04:25:00,-2.262273e+06,218660.421950,69.534304,15.520784,416,0 days 00:05:00,37.704661
2,6,37,203.303419,302.324596,49,5,627,2020-03-13 04:30:00,2020-03-13 04:30:00,-2.263303e+06,219324.595821,69.524381,15.534945,416,0 days 00:10:00,36.472873
3,4,37,230.460885,211.799724,187,5,432,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.290461e+06,128799.724015,69.340584,13.218530,425,0 days 00:00:00,110.156057
4,5,44,231.873681,213.475405,183,5,533,2020-03-13 04:25:00,2020-03-13 04:25:00,-2.291874e+06,130475.405470,69.326868,13.258308,425,0 days 00:05:00,108.487906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6155,299,27,271.955203,288.680268,159,5,23039,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.331955e+06,205680.267926,68.912816,15.040489,16225,0 days 00:05:00,52.916488
6156,300,30,272.843216,289.783693,142,5,23077,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.332843e+06,206783.692918,68.903864,15.065471,16225,0 days 00:10:00,52.910819
6157,298,34,296.371715,217.621195,30,5,23006,2020-03-14 04:50:00,2020-03-14 04:50:00,-2.356372e+06,134621.195276,68.737655,13.269794,16228,0 days 00:00:00,122.630657
6158,299,30,296.984193,220.121651,26,5,23042,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.356984e+06,137121.651293,68.730769,13.329528,16228,0 days 00:05:00,120.888777


In [32]:
# append max min and mean reflectivity values based on each cell id and times to the tracks_closer_AMF1_index_reset dataframe
# tracks_closer_AMF1_index_reset['speed'] = velocity
# tracks_closer_AMF1_index_reset['area1'] = area1
tracks_closer_AMF1_index_reset['mean dBZ'] = pd.Series(mean_reflectivities1)
tracks_closer_AMF1_index_reset['max dBZ'] = pd.Series(max_reflectivities1)

In [33]:
tracks_closer_AMF1_index_reset

,frame,idx,hdim_1,hdim_2,num,threshold_value,feature,time,timestr,projection_y_coordinate,projection_x_coordinate,latitude,longitude,cell,time_cell,distance,mean dBZ,max dBZ
0,4,28,201.219736,300.689233,11,5,423,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.261220e+06,217689.233099,69.544704,15.498959,416,0 days 00:00:00,39.132921,12.845599,38.092506
1,5,38,202.273236,301.660422,35,5,527,2020-03-13 04:25:00,2020-03-13 04:25:00,-2.262273e+06,218660.421950,69.534304,15.520784,416,0 days 00:05:00,37.704661,13.138674,36.584485
2,6,37,203.303419,302.324596,49,5,627,2020-03-13 04:30:00,2020-03-13 04:30:00,-2.263303e+06,219324.595821,69.524381,15.534945,416,0 days 00:10:00,36.472873,13.454442,38.092506
3,4,37,230.460885,211.799724,187,5,432,2020-03-13 04:20:00,2020-03-13 04:20:00,-2.290461e+06,128799.724015,69.340584,13.218530,425,0 days 00:00:00,110.156057,12.845599,38.092506
4,5,44,231.873681,213.475405,183,5,533,2020-03-13 04:25:00,2020-03-13 04:25:00,-2.291874e+06,130475.405470,69.326868,13.258308,425,0 days 00:05:00,108.487906,13.138674,36.584485
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6155,299,27,271.955203,288.680268,159,5,23039,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.331955e+06,205680.267926,68.912816,15.040489,16225,0 days 00:05:00,52.916488,15.240493,41.116443
6156,300,30,272.843216,289.783693,142,5,23077,2020-03-14 05:00:00,2020-03-14 05:00:00,-2.332843e+06,206783.692918,68.903864,15.065471,16225,0 days 00:10:00,52.910819,14.702386,40.106802
6157,298,34,296.371715,217.621195,30,5,23006,2020-03-14 04:50:00,2020-03-14 04:50:00,-2.356372e+06,134621.195276,68.737655,13.269794,16228,0 days 00:00:00,122.630657,15.301029,39.603226
6158,299,30,296.984193,220.121651,26,5,23042,2020-03-14 04:55:00,2020-03-14 04:55:00,-2.356984e+06,137121.651293,68.730769,13.329528,16228,0 days 00:05:00,120.888777,15.240493,41.116443


In [34]:
tracks_closer_AMF1_index_reset.to_csv('../saved-files/threshold-5/final-tracks-threshold-5-excluding-area.csv', index=False)

# END 

# Next Step : 

### Plot the area, max dBZ, mean dBz against time or distance.
## Do they intensify as it approaches the coast
### Perform statistics on cells
## Do the cells that merged/split?

### -----------------------------

## distance to landfall.  (2)
# start with time to landfall --- (1)